In [1]:
!pip install -q transformers datasets wandb

In [2]:
!huggingface-cli login --token hf_

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import torch
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
from transformers import BertConfig, BertModel
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import wandb
import numpy as np

# Initialize wandb
wandb.init(
    project="bert-biencoder-empathy"
)

# Load dataset
dataset = load_dataset("minoosh/EPITOME_pairs2")

# Initialize bi-encoder model (e.g., BERT as a sentence encoder)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name)

# Tokenize both text1 and text2 independently
def preprocess_function(examples):
    text1_encodings = tokenizer(examples['text1'], truncation=True, padding=True, max_length=512)
    text2_encodings = tokenizer(examples['text2'], truncation=True, padding=True, max_length=512)
    return {
        'input_ids_text1': text1_encodings['input_ids'],
        'attention_mask_text1': text1_encodings['attention_mask'],
        'input_ids_text2': text2_encodings['input_ids'],
        'attention_mask_text2': text2_encodings['attention_mask'],
        'labels': examples['label']
    }

# Apply tokenization
tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
tokenized_val = dataset['validation'].map(preprocess_function, batched=True)

# Remove unnecessary columns and set format for PyTorch
columns_to_keep = ['input_ids_text1', 'attention_mask_text1', 'input_ids_text2', 'attention_mask_text2', 'labels']
tokenized_train.set_format(type='torch', columns=columns_to_keep)
tokenized_test.set_format(type='torch', columns=columns_to_keep)
tokenized_val.set_format(type='torch', columns=columns_to_keep)

# Define a custom collator to handle text1 and text2 encoding
class BiEncoderCollator:
    def __call__(self, features):
        # Pad each batch dynamically
        batch = {
            'input_ids_text1': torch.nn.utils.rnn.pad_sequence(
                [torch.tensor(f['input_ids_text1']) for f in features], batch_first=True),
            'attention_mask_text1': torch.nn.utils.rnn.pad_sequence(
                [torch.tensor(f['attention_mask_text1']) for f in features], batch_first=True),
            'input_ids_text2': torch.nn.utils.rnn.pad_sequence(
                [torch.tensor(f['input_ids_text2']) for f in features], batch_first=True),
            'attention_mask_text2': torch.nn.utils.rnn.pad_sequence(
                [torch.tensor(f['attention_mask_text2']) for f in features], batch_first=True),
            'labels': torch.tensor([f['labels'] for f in features], dtype=torch.float)
        }
        '''batch = {
            'input_ids_text1': torch.stack([f['input_ids_text1'] for f in features]),
            'attention_mask_text1': torch.stack([f['attention_mask_text1'] for f in features]),
            'input_ids_text2': torch.stack([f['input_ids_text2'] for f in features]),
            'attention_mask_text2': torch.stack([f['attention_mask_text2'] for f in features]),
            'labels': torch.tensor([f['labels'] for f in features], dtype=torch.float)
        }'''
        return batch

collator = BiEncoderCollator()

# Define the compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()
    labels = labels.squeeze()

    mse = mean_squared_error(labels, predictions)
    mae = mean_absolute_error(labels, predictions)
    pearson_corr, _ = pearsonr(predictions, labels)
    spearman_corr, _ = spearmanr(predictions, labels)
    cosine_sim = torch.nn.functional.cosine_similarity(torch.tensor(predictions), torch.tensor(labels), dim=0).mean().item()

    return {
        "mse": mse,
        "mae": mae,
        "pearson_corr": pearson_corr,
        "spearman_corr": spearman_corr,
        "cosine_sim": cosine_sim  # Optional metric for similarity tasks
    }

# Define a custom BiEncoder model
class BiEncoderModel(torch.nn.Module):
    def __init__(self, base_model, config=None, loss_fn="mse"):
        super(BiEncoderModel, self).__init__()
        self.base_model = base_model
        self.cos = torch.nn.CosineSimilarity(dim=1)
        self.loss_fn = loss_fn
        self.config = config

    def forward(self, input_ids_text1, attention_mask_text1, input_ids_text2, attention_mask_text2, labels=None):
        # Encode text1 and text2 separately
        outputs_text1 = self.base_model(input_ids_text1, attention_mask=attention_mask_text1)
        outputs_text2 = self.base_model(input_ids_text2, attention_mask=attention_mask_text2)

        # Extract [CLS] token embeddings (first token)
        cls_embedding_text1 = outputs_text1.last_hidden_state[:, 0, :]
        cls_embedding_text2 = outputs_text2.last_hidden_state[:, 0, :]

        # Calculate cosine similarity between the two embeddings
        cos_sim = self.cos(cls_embedding_text1, cls_embedding_text2)

        loss = None
        if labels is not None:
            if self.loss_fn == "mse":
                loss_fct = torch.nn.MSELoss()  # Mean Squared Error Loss
            elif self.loss_fn == "mae":
                loss_fct = torch.nn.L1Loss()  # Mean Absolute Error Loss
            elif self.loss_fn == "contrastive":
                loss_fct = self.contrastive_loss
            elif self.loss_fn == "cosine_embedding":
                loss_fct = torch.nn.CosineEmbeddingLoss()  # Cosine Embedding Loss

            if self.loss_fn == "cosine_embedding":
                labels_cosine = 2 * (labels > 0.5).float() - 1  # Convert labels to binary for cosine embedding loss
                loss = loss_fct(cls_embedding_text1, cls_embedding_text2, labels_cosine)
            else:
                loss = loss_fct(cos_sim, labels)

        return {"loss": loss, "logits": cos_sim}

    def contrastive_loss(self, cos_sim, labels, margin=0.5):
        loss = torch.mean((1 - labels) * torch.pow(cos_sim, 2) + labels * torch.pow(torch.clamp(margin - cos_sim, min=0.0), 2))
        return loss

# Initialize the Bi-Encoder model with a specific loss function
def train_biencoder(loss_fn):
    # Load pre-trained BERT configuration and model
    config = BertConfig.from_pretrained(model_name)
    bert_model = BertModel.from_pretrained(model_name)

    # Initialize your custom BiEncoderModel with the BERT model and config
    bi_encoder_model = BiEncoderModel(base_model=bert_model, config=config, loss_fn=loss_fn)
    #bi_encoder_model = BiEncoderModel(base_model, loss_fn)

    # Define TrainingArguments
    training_args = TrainingArguments(
        output_dir=f"./output/empathy-biencoder-{loss_fn}_Ds2",
        evaluation_strategy="epoch",    # Evaluate at the end of each epoch
        logging_dir='./logs',           # Directory for logs
        logging_steps=10,               # Log every 10 steps
        per_device_train_batch_size=wandb.config['batch_size'],
        per_device_eval_batch_size=wandb.config['batch_size'],
        num_train_epochs=wandb.config['epochs'],
        warmup_steps=100,
        learning_rate=wandb.config['learning_rate'],
        weight_decay=0.01,
        report_to="wandb",
        save_strategy="epoch",          # Save checkpoints at the end of each epoch
        load_best_model_at_end=True,
        push_to_hub=True,
        save_total_limit=2              # Keep only the 2 most recent checkpoints
    )

    # Define the Trainer
    trainer = Trainer(
        model=bi_encoder_model,             # Custom BiEncoder model
        args=training_args,                 # Training arguments
        train_dataset=tokenized_train,      # Training dataset
        eval_dataset=tokenized_val,         # Validation dataset
        data_collator=collator,             # Custom collator for handling bi-encoder inputs
        compute_metrics=compute_metrics     # Function to compute metrics
    )

    # Train the model
    trainer.train()

    # Evaluate on the test set
    trainer.evaluate(tokenized_test)

    # Save the model to Hugging Face Hub
    trainer.save_model(f"./output/empathy-biencoder-{loss_fn}_Ds2")
    trainer.push_to_hub(f"minoosh/empathy-biencoder-{loss_fn}_Ds2")

    # Finish wandb run
    wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


README.md:   0%|          | 0.00/589 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/660k [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Map:   0%|          | 0/2467 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

Map:   0%|          | 0/309 [00:00<?, ? examples/s]

In [4]:
# Train bi-encoder with different loss functions
loss_functions = ["mse", "mae", "contrastive", "cosine_embedding"]
loss_fn = loss_functions[1]
wandb.init(project="bert-biencoder-empathy", name=f"bert-biencoder-empathy-{loss_fn}_Ds2", config={"epochs": 5, "batch_size": 16, "learning_rate": 2e-5})
train_biencoder(loss_fn)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/tmp/ipykernel_30/3833924472.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['input_ids_text1']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor

Epoch,Training Loss,Validation Loss,Mse,Mae,Pearson Corr,Spearman Corr,Cosine Sim
1,0.153100,0.159352,0.041124,0.159252,0.352507,0.360193,0.840801
2,0.127200,0.152880,0.036222,0.152746,0.516035,0.516792,0.868463
3,0.101500,0.139206,0.029945,0.139116,0.558192,0.562953,0.874459
4,0.098300,0.148448,0.034032,0.148342,0.564564,0.567231,0.878567
5,0.086500,0.147851,0.033814,0.147728,0.564612,0.569424,0.877721


/tmp/ipykernel_30/3833924472.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['input_ids_text1']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['attention_mask_text1']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['input_ids_text2']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:58: UserWarning: To copy construct f

No files have been modified since last commit. Skipping to prevent empty commit.


eval/cosine_sim,▁▆▇██▇
eval/loss,█▆▃▅▅▁
eval/mae,█▆▃▅▅▁
eval/mse,█▅▁▄▃▁
eval/pearson_corr,▁▆████
eval/runtime,▁▂▂▂█▅
eval/samples_per_second,█▇▇▇▁▄
eval/spearman_corr,▁▆████
eval/steps_per_second,█▇▇▇▁▄
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████


In [5]:
# Train bi-encoder with different loss functions
loss_functions = ["mse", "mae", "contrastive", "cosine_embedding"]
loss_fn = loss_functions[2]
wandb.init(project="bert-biencoder-empathy", name=f"bert-biencoder-empathy-{loss_fn}_Ds2", config={"epochs": 5, "batch_size": 16, "learning_rate": 2e-5})
train_biencoder(loss_fn)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/tmp/ipykernel_30/3833924472.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['input_ids_text1']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor

Epoch,Training Loss,Validation Loss,Mse,Mae,Pearson Corr,Spearman Corr,Cosine Sim
1,0.054000,0.062897,0.042491,0.168469,0.233082,0.253217,0.769336
2,0.050400,0.055750,0.036857,0.157270,0.360932,0.355529,0.816929
3,0.048300,0.055407,0.035690,0.155001,0.413554,0.404821,0.824564
4,0.045200,0.054082,0.035762,0.155291,0.430628,0.423020,0.829136
5,0.044800,0.053582,0.036198,0.156072,0.433215,0.428420,0.828967


/tmp/ipykernel_30/3833924472.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['input_ids_text1']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['attention_mask_text1']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['input_ids_text2']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:58: UserWarning: To copy construct f

No files have been modified since last commit. Skipping to prevent empty commit.


eval/cosine_sim,▁▆▇▇▇█
eval/loss,█▃▃▂▂▁
eval/mae,█▄▄▄▄▁
eval/mse,█▃▂▂▃▁
eval/pearson_corr,▁▅▇███
eval/runtime,▃▃▂▂▁█
eval/samples_per_second,▆▇▇▇█▁
eval/spearman_corr,▁▄▆▇▇█
eval/steps_per_second,▆▆▇▇█▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████


In [5]:
# Train bi-encoder with different loss functions
loss_functions = ["mse", "mae", "contrastive", "cosine_embedding"]
loss_fn = loss_functions[3]
wandb.init(project="bert-biencoder-empathy", name=f"bert-biencoder-empathy-{loss_fn}_Ds2", config={"epochs": 5, "batch_size": 16, "learning_rate": 2e-5})
train_biencoder(loss_fn)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/tmp/ipykernel_30/3833924472.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['input_ids_text1']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor

Epoch,Training Loss,Validation Loss,Mse,Mae,Pearson Corr,Spearman Corr,Cosine Sim
1,0.100500,0.110309,0.126254,0.295663,0.093024,0.065044,0.109258
2,0.085200,0.084715,0.134844,0.315290,0.080329,0.092797,-0.075253
3,0.060300,0.072980,0.143082,0.328031,0.122360,0.102984,-0.172735
4,0.065500,0.072590,0.132073,0.316032,0.183973,0.182827,-0.091000
5,0.085300,0.071080,0.138098,0.324282,0.184668,0.179811,-0.138900


/tmp/ipykernel_30/3833924472.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['input_ids_text1']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['attention_mask_text1']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(f['input_ids_text2']) for f in features], batch_first=True),
/tmp/ipykernel_30/3833924472.py:58: UserWarning: To copy construct f

No files have been modified since last commit. Skipping to prevent empty commit.


eval/cosine_sim,█▅▃▄▄▁
eval/loss,█▃▁▁▁▁
eval/mae,▁▅▇▅▇█
eval/mse,▁▄▇▃▅█
eval/pearson_corr,▂▁▄██▄
eval/runtime,▁▁▁▁▁█
eval/samples_per_second,█████▁
eval/spearman_corr,▁▃▃██▆
eval/steps_per_second,█████▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
